In [2]:
import os
import random
import time

import numpy as np
import torch

from func import out_string_nets, textread
from mashup import load_multi, mashup, mashup_multi

torch.manual_seed(1)
random.seed(1)
np.random.seed(1)

# Load config

In [ ]:
# wrtie some function to load config

org = args.org
net = args.net
method = args.method


mixup = True
ndim = args.ndim

torch_thread = args.torch_thread
num_thread = args.num_thread

# Load data

In [ ]:
string_nets = out_string_nets(net, org)

network_files = []
for i in range(len(string_nets)):
    network_files.append(
        f'data/networks/{org}/{org}_string_{string_nets[i]}_adjacency.txt')

# Load gene list
gene_file = f'data/networks/{org}/{org}_{net}_genes.txt'
genes = textread(gene_file)
ngene = len(genes)

# Gemini

In [ ]:
# Network integration
# not finished 

start_time = time.time()
print(f'{method}_{org}_{net}_{ndim}')
print('[Mashup]')

embd_name = f'data/embed/{method}_{org}_{net}_{ndim}'

node_weights = None

weights = np.zeros(len(network_files))
embed_type = args.embed_type:
axis = 1
separate = np.load(
    f'data/separate/{net}_{org}_type0_' +
    f'{embed_type}{axis}_{args.cluster_method}_' +
    f'{args.level}.npy')
if args.weight == 2:
    clus_count = np.ones(len(set(separate)))
elif args.weight == 1:
    clus_count = np.zeros(len(set(separate)))
separate = separate[:len(network_files)]
for i in separate:
    clus_count[i] += 1
if args.weight == 2:
    clus_weight = 1/clus_count + \
        args.ori_weight/len(network_files)
elif args.weight == 1:
    clus_weight = 1/clus_count
weights += np.array([clus_weight[i] for i in separate])


embd_name += f'_{args.embed_type}{args.axis}_' + \
    f'separate{args.separate}_{args.cluster_method}' + \
    f'_weight{args.weight}_{args.ori_weight}'
args.separate = None
print(args.level)
embd_name += f'_{args.level}'

if args.mixup > 0:
    network_pairs_mixup_ = []
    from numpy.random import choice
    random.seed(1)
    print(args.ndim)
    # np.random.seed(1)
    p = weights
    p = p/p.sum()
    list_of_candidates = np.arange(len(network_files))
    for idd in range(args.mixup):
        network_pairs_mixup = []
        args.ori_seed = int(np.floor(args.ori_seed*10000)/10000)
        np.random.seed(idd+args.ori_seed)
        for _ in range(round(len(network_files)*args.mixup2)):
            # for ixd in range(args.mixup):
            draw = choice(list_of_candidates, 2,
                          p=p)
            d0, d1 = draw[0], draw[1]
            # if separate[d0] != separate[d1]:
            n0 = network_files[d0]
            n1 = network_files[d1]
            network_pairs_mixup.append([n0, 1, n1, 1])
        network_pairs_mixup_.append(network_pairs_mixup)
    mixup = 'mixup'
    embd_name += f'_mixup{args.mixup}_{args.mixup2}'
    embd_name += f'_gamma{args.gamma}'
    network_files_all = network_pairs_mixup_